<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/echo/echo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This test is designed to hammer Cytoscape by sending a long sequence of short operations. On its own, it's an effective test of Jupyter-Bridge -- it verifies that a Colab Notebook can keep a reliable channel to a private Cytoscape.

But when this test is executed in multiple Colabs (each connected to its own Cytoscape in its own VM), it verifies that Jupyter-Bridge can process a high volume of requests while keeping good track of requests and replies.


---
# Setup data files, py4cytoscape and Cytoscape connection
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

## Setup: Fetch latest py4cytoscape




**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.12').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [ ]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.6.0' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  Created wheel for python-igraph: filename=python_igraph-0.9.11-py3-none-any.whl size=9074 sha256=616063ea1ebde861b4ac2fdb240f7a58473ac85449152fe6b3140a8fe80929ef
  Stored in directory: /root/.cache/pip/wheels/7d/e4/0c/0c83f70bd0b99ce8aea47a21f8e52502169e7dd17808d12f30
Successfully built python-igraph
NumExpr defaulting to 2 threads.
Loading Javascript client ... 6873edeb-7ad8-478f-a92d-da30e10c344d on https://jupyter-bridge.cytoscape.org
ADVICE: WHEN RUNNING UNDER COLAB, DO NOT RE-RUN THIS CELL WITHOUT MANUALLY EXECUTING Runtime | Factory Reset Runtime FROM THE COLAB MENU FIRST.


<IPython.core.display.Javascript object>

## Setup: Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [ ]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'automationAPIVersion': '1.5.0',
 'cytoscapeVersion': '3.9.1',
 'jupyterBridgeVersion': '0.0.2',
 'py4cytoscapeVersion': '1.3.0'}

## Setup: Initialize a basic static message
The basic message is a random digit sequence that is unlikely to be used by any other Colab Notebook instance.

In [ ]:
import uuid as uu
my_uuid = str(uu.uuid4())

print(f'Fixed message: {my_uuid}')


rm: cannot remove 'output/': No such file or directory
total 8
drwxr-xr-x 2 root root 4096 Jun 28 22:23 logs
drwxr-xr-x 1 root root 4096 Jun 15 13:42 sample_data


## Test
Call Cytoscape's echo command with a unique message and get back the unique message.

The message consists of a random uuid (previously generated) combined with a monotonically increasing value.

Thus messages sent from this Colab instance are guaranteed to be unique in the universe, and each message sent from this Colab instance is guaranteed to be different from any previous messages.


In [ ]:
import py4cytoscape as p4c

for i in range(0, 10):
  message = f'{i}: {my_uuid}'
  ret = p4c.command_echo(message)
  if message == ret:
    print(f'Successful: {i}')
  else:
    print(f'Failure: got {ret}, expected {message}')


Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has 166981992 bytes
